In [5]:
!pip install sqlalchemy

In [7]:
!pip install pymysql

In [9]:
pip show sqlalchemy | grep Version

Version: 2.0.34
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

 ········


In [9]:
# Define database and connection string for Sakila
bd = "sakila" 
password = ""  # Replace with actual password
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
    
try:
    # Create engine
    engine = create_engine(connection_string)
    
    # Test the connection
    with engine.connect() as connection:
        print("Successfully connected to the Sakila database!")
        
        # Test query to show tables
        query = "SHOW TABLES;"
        tables = pd.read_sql(query, connection)
        print("\
Available tables in the Sakila database:")
        print(tables)

except Exception as e:
    print("Error connecting to the database:", str(e))

Successfully connected to the Sakila database!
Available tables in the Sakila database:
              Tables_in_sakila
0                        actor
1                   actor_info
2                      address
3                     category
4                         city
5                      country
6                     customer
7                customer_list
8      customer_rental_summary
9                         film
10                  film_actor
11               film_category
12                   film_list
13                   film_text
14                   inventory
15                    language
16  nicer_but_slower_film_list
17                     payment
18                      rental
19      sales_by_film_category
20              sales_by_store
21                       staff
22                  staff_list
23                       store


In [11]:
def rentals_month(engine, month, year):
    """
    Retrieve rental data for a specific month and year from the Sakila film database.

    Parameters:
    engine (sqlalchemy.engine.base.Engine): Database connection engine.
    month (int): Month for which rental data is to be retrieved.
    year (int): Year for which rental data is to be retrieved.

    Returns:
    pandas.DataFrame: Rental data for the specified month and year.
    """
    query = f"""
    SELECT customer_id, COUNT(*) as rentals_count
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    GROUP BY customer_id;
    """

    with engine.connect() as connection:
        data = pd.read_sql(query, connection)
    return data

In [13]:
def rental_count_month(df, month, year):
    """
    Calculate the number of rentals per customer for a specific month and year.
    
    Parameters:
    df: pandas DataFrame from rentals_month function
    month: int, month number (1-12)
    year: int, year
    
    Returns:
    pandas DataFrame with customer_id and their rental count for the specified month/year
    """
    # Create column name based on month and year
    column_name = f"sakila_rentals_{str(month).zfill(2)}_{year}"
    
    # Group by customer_id and count rentals
    rental_counts = df.groupby('customer_id')['rental_id'].count().reset_index()
    
    # Rename count column to reflect month and year
    rental_counts = rental_counts.rename(columns={'rental_id': column_name})
    
    return rental_counts

print("Function updated and ready for real data from Sakila database.")

Function updated and ready for real data from Sakila database.


In [15]:
# Updated function without mock data
def compare_rentals(df1, df2):
    """
    Compare the number of rentals made by each customer in two different months/years.

    Parameters:
    df1: pandas DataFrame containing rental counts for the first month/year
    df2: pandas DataFrame containing rental counts for the second month/year

    Returns:
    pandas DataFrame with customer_id, rental counts for both months/years, and the difference
    """
    # Merge the two DataFrames on customer_id
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2'))

    # Fill NaN values with 0 (in case some customers are missing in one of the months)
    combined_df = combined_df.fillna(0)

    # Extract column names for rental counts
    rental_col1 = combined_df.columns[1]  # First rental column
    rental_col2 = combined_df.columns[2]  # Second rental column

    # Calculate the difference and add a new column
    combined_df['difference'] = combined_df[rental_col1] - combined_df[rental_col2]

    return combined_df

print("Function updated and ready for real data from Sakila database.")

Function updated and ready for real data from Sakila database.
